In [16]:
from langchain_community.document_loaders import PyPDFLoader

document = PyPDFLoader("rag.pdf")

loader = document.lazy_load()

text = ""

for page in loader:
    text += page.page_content


In [ ]:
from langchain_text_splitters import CharacterTextSplitter


text_splitter = CharacterTextSplitter(
    chunk_size=2500,
    chunk_overlap=100,
    separator="\n" 
)

texts = text_splitter.create_documents([text])

#embedding echo por nomic-embed-text 

In [46]:
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model="nomic-embed-text"
)

#embedding echo por todo-minilm

In [6]:
from langchain_chroma import Chroma 


vector_store = Chroma(
    collection_name="test",
    embedding_function=embedding,
    persist_directory="./vectorStore-nomic"
)

In [48]:
vector_store.add_documents(texts)

['248c0941-3395-411a-8f74-893c107c8ac5']

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os 
load_dotenv()

api_key = os.getenv("API_KEY")

llm = ChatGoogleGenerativeAI(
    api_key=api_key,
    model = "gemini-2.5-flash",
    temperature=0.5
)  

In [50]:
from langchain_core.prompts import ChatPromptTemplate

ai_msg = [""]
human_msg = [ ]

prompt_template = ChatPromptTemplate([
    ("system","""
Eres un asistente encargado de responder preguntas sobre
Langchain.
Responde solo si {context} posee contenido. Si
el contexto esta vacio,
responde "No tengo suficiente informacion para
reponder esa pregunta"
"""),
("ai","{ai_msg}"),
("human","{human_msg}")
])

In [51]:
input_user =input("Human: ")
human_msg.append(input_user)

In [ ]:
docs = vector_store.similarity_search(input_user, k=10,
                                         filter={"page": 10} )

prompt= prompt_template.invoke({
    "context":docs,
    "ai_msg":ai_msg,
    "human_msg":human_msg})

In [55]:
response = llm.invoke(prompt)
print(response.content)



La tecnología RAG (Generación Aumentada por Recuperación) aporta varios beneficios a los esfuerzos de la IA generativa de una organización:

*   **Implementación rentable**: Es un enfoque más económico para introducir nuevos datos en un LLM sin necesidad de volver a entrenar el modelo fundacional, haciendo la IA generativa más accesible.
*   **Información actual**: Permite a los desarrolladores proporcionar las últimas investigaciones, estadísticas o noticias a los modelos generativos, conectando el LLM a fuentes de información que se actualizan con frecuencia.
*   **Mayor confianza de los usuarios**: El LLM puede presentar información precisa con atribución de la fuente, incluyendo citas o referencias, lo que aumenta la confianza del usuario.
*   **Más control para los desarrolladores**: Los desarrolladores pueden probar y mejorar sus aplicaciones de chat de manera más eficiente, controlando y cambiando las fuentes de información, restringiendo la recuperación de datos confidenciales 